In [1]:
import pandas as pd
import numpy as np

In [2]:
# https://www.kaggle.com/datasets/martj42/international-football-results-from-1872-to-2017?resource=download
df = pd.read_csv("../data/results.csv")

In [3]:
df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [4]:
df["home_match"] = (df["home_team"]==df["country"]).astype(int)

In [5]:
df["result"] = np.where(df["home_score"]>df["away_score"], 1, np.where(df["home_score"]<df["away_score"], -1, 0))

In [6]:
wm_teams = {
    "Qatar": "QAT", 
    "Ecuador":"ECU", 
    "Senegal":"SEN", 
    "Netherlands":"NED",
    "England":"ENG",
    "Iran":"IRN", 
    "United States":"USA", 
    "Wales":"EUR",
    "Argentina":"ARG", 
    "Saudi Arabia":"KSA", 
    "Mexico":"MEX", 
    "Poland":"POL",
    "France":"FRA", 
    "Peru":"ICP-1", 
    "Denmark":"DEN", 
    "Tunisia":"TUN",
    "Spain":"ESP", 
    "New Zealand":"ICP-2", 
    "Germany":"GER", 
    "Japan":"JPN",
    "Belgium":"BEL", 
    "Canada":"CAN", 
    "Morocco":"MAR", 
    "Croatia":"CRO",
    "Brazil":"BRA", 
    "Serbia":"SRB", 
    "Switzerland":"SUI", 
    "Cameroon":"CMR",
    "Portugal":"POR", 
    "Ghana":"GHA", 
    "Uruguay":"URU", 
    "South Republic":"KOR"
}

In [7]:
df["home_team"] = df["home_team"].map(wm_teams)
df["away_team"] = df["away_team"].map(wm_teams)

In [8]:
df = df.dropna()

In [9]:
df["date"] = pd.to_datetime(df["date"])

In [10]:
df = df[df["date"].dt.year >= 1922].reset_index(drop=True)

In [11]:
df["result"].value_counts()

 1    1742
-1    1053
 0     928
Name: result, dtype: int64

In [12]:
df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,home_match,result
0,1922-01-15,FRA,BEL,2,1,Friendly,Colombes,France,False,1,1
1,1922-03-13,ENG,EUR,1,0,British Championship,Liverpool,England,False,1,1
2,1922-03-26,BEL,NED,4,0,Friendly,Antwerp,Belgium,False,1,1
3,1922-03-26,GER,SUI,2,2,Friendly,Frankfurt am Main,Germany,False,1,0
4,1922-04-15,BEL,DEN,0,0,Friendly,Liège,Belgium,False,1,0


In [13]:
###  Elo score from 
# https://medium.com/mlearning-ai/how-to-calculate-elo-score-for-international-teams-using-python-66c136f01048

In [14]:
confederation_tournaments=['AFC Asian Cup','African Cup of Nations','UEFA Euro','Copa América','CONCACAF Championship','Oceania Nations Cup']

def k_value(tournament):
    k=5
    if tournament == 'Friendly':
        k=10
    elif tournament == 'FIFA World Cup qualification':
        k=25
    elif tournament in confederation_tournaments:
        k=40
    elif tournament == 'FIFA World Cup':
        k=55
    return k
    
def expected_result(loc,aw):
    dr=loc-aw
    we=(1/(10**(-dr/400)+1))
    return [np.round(we,3),1-np.round(we,3)]

def actual_result(loc,aw):
    if loc<aw:
        wa=1
        wl=0
    elif loc>aw:
        wa=0
        wl=1
    elif loc==aw:
        wa=0.5
        wl=0.5
    return [wl,wa]

def calculate_elo(elo_l,elo_v,local_goals,away_goals,tournament):
    
    k=k_value(tournament)
    wl,wv=actual_result(local_goals,away_goals)
    wel,wev=expected_result(elo_l,elo_v)

    elo_ln=elo_l+k*(wl-wel)
    elo_vn=elo_v+k*(wv-wev)

    return elo_ln,elo_vn

In [ ]:
current_elo={}
for idx,row in df.iterrows():
    
    local=row['home_team']
    away=row['away_team']
    local_goals=row['home_score']
    away_goals=row['away_score']
    tournament = row['tournament']
    

    if local not in current_elo.keys():
        current_elo[local]=1300
    
    if away not in current_elo.keys():
        current_elo[away]=1300
    
    elo_l=current_elo[local]
    elo_v=current_elo[away]
    elo_ln,elo_vn=calculate_elo(elo_l,elo_v,local_goals,away_goals,tournament)

    current_elo[local]=elo_ln
    current_elo[away]=elo_vn
    
    df.loc[idx,'Elo_h_after']=elo_ln
    df.loc[idx,'Elo_a_after']=elo_vn 
    df.loc[idx,'Elo_h_before']=elo_l
    df.loc[idx,'Elo_a_before']=elo_v

In [ ]:
df.head()

In [ ]:
df = df.rename(columns={"Elo_h_before":"home_elo", "Elo_a_before":"away_elo"})

In [ ]:
df[
    ["date","home_team","away_team","home_score","away_score","away_elo","home_elo","tournament","home_match","result"]
].to_pickle("../data/results_prep.pkl")